In [10]:
def keep_record(model, xtest, ytest):
    predictions = model.predict(xtest)
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(ytest, axis=1)
    precision, recall, fscore, support = score(y_true, y_pred)
    precisions.append(precision)
    recalls.append(recall)
    fscores.append(fscore)
    supports.append(support)
    
    cm = metrics.confusion_matrix(y_true, y_pred)
    cms.append(cm)


    scores = model.evaluate(xtest, ytest, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import metrics

num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)

cvscores = []
histories = []
precisions = []
recalls = []
fscores = []
supports = []
cms = []
    
epochs = 10
batch_size = 64

for train, test in kfold.split(X, Y.argmax(1)): 
    model = get_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    history = model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=1)
    histories.append(history)
    keep_record(model, X[test], Y[test])


print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [16]:
tp = 0
tn = 0
fp = 0
fn = 0
for cm in cms:
    tn = tn + cm[0][0]
    fn = fn + cm[1][0]
    tp = tp + cm[1][1]
    fp = fp + cm[0][1]
    
precision = (tp) / (tp + fp)
recall = (tp) / (tp + fn)
f1 = (2 * precision * recall) / (precision + recall)
acc = (tp + tn) / (tp + tn + fp + fn)

print("precision", precision )
print("recall", recall )
print("f1", f1 )
print("acc", acc )